In [50]:
import json
import pandas as pd

# Load JSON
with open("result.json") as f:
    data = json.load(f)

# Convert dict of dicts into a DataFrame
records = []
for id_, details in data.items():
    flat = {'ID': id_}
    flat.update(details)
    records.append(flat)

df = pd.DataFrame(records)
df

,ID,Dashboard,Position,Name,Email,LIS,EQ,Critical Skills,Critical Skills Scores,Necessary Skills,Necessary Skills Scores,Beneficial Skills,Beneficial Skills Scores,Key skills mean,Necessary skills mean,beneficial skills mean
0,LDP0394,D12: Corporate Communications and Training,CORPORATE CULTURE & TRAINING MANAGER,Donald Finchum,d.finchum31@anondomain.com,90.5050,131,"[Communication Skills, Training & Onboarding, ...","[96, 90, 98, 97]","[Organization Skills, Measuring & Evaluating R...","[98, 100, 89, 86, 88]","[Social Insight, Supportiveness, Charisma, Emo...","[100, 97, 100, 96, 83]",95.25,92.20,95.2
1,LDP4315,D4: Logistics and Site Management,SITE MANAGER - CIVIL,Susan Clark,s.clark59@anondomain.com,73.3140,107,"[Problem-Solving, Project Management, Organiza...","[90, 93, 81, 68, 68]","[Monitoring Employee Performance, Measuring & ...","[78, 60, 83, 87]","[Negotiating Ability, Resilience/Hardiness, In...","[85, 62, 69, 39, 53]",80.00,77.00,61.6
2,LDP0127,D6: Procurement and Supply Chain Management,PROCUREMENT MANAGER-OFFICE SUPPLY,John Mendoza,j.mendoza41@anondomain.com,76.2050,110,"[Problem-Solving, Project Management, Informat...","[80, 93, 86, 68, 63]","[Conflict Management, Change Management, Commu...","[79, 80, 72, 95]","[Innovative Mindset, Resilience/Hardiness, Sup...","[84, 68, 100, 64, 89]",78.00,81.50,81.0
3,LDP4802,D5: Engineering and Technical Management,COMMISSIONING MANAGER,Juan Spegal,j.spegal48@anondomain.com,75.4210,109,"[Problem-Solving, Project Management, Communic...","[100, 93, 82, 67, 64]","[Information Gathering & Analysis, Measuring &...","[64, 76, 77, 81]","[Innovative Mindset, Negotiating Ability, Resi...","[93, 66, 86, 75, 92]",81.20,74.50,82.4
4,LDP1313,D13: Administrative and Support Services,SUPPORT SERVICES MANAGER,Ruth Kingsley,r.kingsley18@anondomain.com,81.1940,114,"[Organization Skills, Problem-Solving, Communi...","[93, 100, 88, 77]","[Project Management, Customer Orientation, Con...","[91, 93, 70, 69, 75]","[Supportiveness, Patience, Negotiating Ability...","[90, 76, 91, 100, 89]",89.50,79.60,89.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,LDP3000,D3: Senior Management - Operations and Logistics,PROJECT MANAGER-CIVIL,Deborah Woll,d.woll16@anondomain.com,57.0325,67,"[Problem-Solving, Project Management, Communic...","[50, 68, 60, 80, 97]","[Information Gathering & Analysis, Measuring &...","[56, 54, 59, 64]","[Organization Skills, Negotiating Ability, Res...","[68, 69, 46, 74, 93]",71.00,58.25,70.0
77,LDP3019,D3: Senior Management - Operations and Logistics,PROJECT MANAGER-INDUSTRIAL,Robert Shirk,r.shirk38@anondomain.com,76.5470,105,"[Problem-Solving, Project Management, Communic...","[70, 98, 86, 83, 73]","[Information Gathering & Analysis, Measuring &...","[94, 56, 83, 93]","[Organization Skills, Negotiating Ability, Res...","[90, 90, 76, 84, 84]",82.00,81.50,84.8
78,LDP4011,D4: Logistics and Site Management,SITE MANAGER-ELECTRICAL,Monique Gomez,m.gomez13@anondomain.com,85.6640,119,"[Problem-Solving, Project Management, Organiza...","[94, 95, 100, 72, 92]","[Monitoring Employee Performance, Measuring & ...","[100, 100, 86, 86]","[Negotiating Ability, Resilience/Hardiness, In...","[83, 86, 94, 90, 100]",90.60,93.00,90.6
79,LDP5422,D5: Engineering and Technical Management,CONSTRUCTION MANAGER-MEP,Michael Trowell,m.trowell14@anondomain.com,80.4350,114,"[Problem-Solving, Project Management, Communic...","[100, 96, 96, 73, 71]","[Information Gathering & Analysis, Measuring &...","[96, 76, 71, 83]","[Innovative Mindset, Negotiating Ability, Resi...","[85, 66, 100, 76, 93]",87.20,81.50,84.0


In [17]:
from langchain_experimental.agents import create_pandas_dataframe_agent
from langchain_google_genai import ChatGoogleGenerativeAI

# LLM
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro-latest")

# Create agent with code execution allowed
agent = create_pandas_dataframe_agent(
    llm,
    df,
    verbose=True,
    allow_dangerous_code=True  
)

# Run your query
agent.run("Who got the lowest EQ?")




> Entering new AgentExecutor chain...
Thought: I need to find the minimum value in the 'EQ' column and the corresponding name.
Action: python_repl_ast
Action Input: 
```python
print(df.loc[df['EQ'] == df['EQ'].min()][['Name', 'EQ']])
```             Name  EQ
76  Deborah  Woll  67
Deborah Woll got the lowest EQ.

Final Answer: Deborah Woll got the lowest EQ.

> Finished chain.


'Deborah Woll got the lowest EQ.'

In [49]:
from langchain_experimental.agents import create_pandas_dataframe_agent
from langchain_google_genai import ChatGoogleGenerativeAI

class LeadershipAnalyticsAgent:
    def __init__(self, df, verbose=True):
        self.llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro-latest")
        self.agent = create_pandas_dataframe_agent(
            self.llm,
            df,
            verbose=verbose,
            allow_dangerous_code=True
        )

    def ask(self, query: str) -> str:
        prompt = (
            "You are a data analytics assistant. You work with structured leadership performance data. "
            "Use Python and Pandas to perform accurate numeric comparisons and filtering. "
            "Avoid guessing. Always return the actual value found in the data.\n\n"
            f"Question: {query}"
        )
        return self.agent.run(prompt)


In [51]:
# Assuming df is already loaded and prepped
agent = LeadershipAnalyticsAgent(df)

# Example questions
#print(agent.ask("Who got the lowest EQ?"))
#print(agent.ask("Which dashboard has the highest average LIS?"))
print(agent.ask("List the top 3 performers in Key skills mean."))




> Entering new AgentExecutor chain...
Thought: I need to sort the dataframe by 'Key skills mean' in descending order and then get the top 3 rows.

Action: python_repl_ast
Action Input:
```python
print(df.sort_values('Key skills mean', ascending=False).head(3))
```         ID                                         Dashboard  \
61  LDP1111  D11: Corporate Strategy and Business Development   
11  LDP8111                  D8: Health, Safety, and Security   
17  LDP1001                   D10: Financial and Risk Control   

                                             Position              Name  \
61  DIRECTOR - BUSINESS DEVELOPMENT & STRATEGIC PA...  William  Smullen   
11                            SR. IT SECURITY MANAGER   Laticia  Bridge   
17                           ACCOUNTS PAYABLE MANAGER      James  Lobur   

                         Email     LIS   EQ  \
61  w.smullen81@anondomain.com  92.615  129   
11   l.bridge31@anondomain.com  86.511  117   
17    j.lobur69@anondomain.com 

In [13]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 20})


In [14]:
from langchain.chains import StuffDocumentsChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain as CombineStuffDocumentsChain

# 🧠 Custom prompt
context_prompt = PromptTemplate.from_template("""
You are a data assistant helping analyze leadership assessment data.

Use the provided context to answer the question below. The context includes information on individuals, dashboards, and their skills.

Context:
{context}

Question:
{question}

Answer by combining insights across individuals. Focus on patterns, averages, and summaries. If no useful context is found, say "Not enough information."
""")

# 🔗 Chain
llm_chain = LLMChain(llm=llm, prompt=context_prompt)

stuff_chain = CombineStuffDocumentsChain(
    llm_chain=llm_chain,
    document_variable_name="context"
)

# 🧠 Updated RAG with context + memory
rag_chain = RetrievalQA(
    retriever=retriever,
    combine_documents_chain=stuff_chain,
    memory=memory
)

# -----------------------------
# Pandas Agent Setup
# -----------------------------
pandas_agent = create_pandas_dataframe_agent(
    llm=llm,
    df=df,
    verbose=True,
    allow_dangerous_code=True
)

# -----------------------------
# Classifier to Route Query
# -----------------------------
classifier_prompt = PromptTemplate.from_template("""
You are a smart classifier. Given a user question, decide if it should be handled using:

- "structured" → if it's about rankings, filters, math, comparisons, scores
- "semantic" → if it's about summaries, meaning, types of skills, descriptions

Return ONLY the word "structured" or "semantic".

Question: {query}
""")

classifier_chain = LLMChain(llm=llm, prompt=classifier_prompt)

# -----------------------------
# Unified Ask Function
# -----------------------------
def ask(query: str) -> str:
    route = classifier_chain.run({"query": query}).strip().lower()
    print(f"[Routing → {route}]")
    
    if route == "structured":
        return pandas_agent.run(query)
    elif route == "semantic":
        return rag_chain.run(query)
    else:
        return "Sorry, I couldn't confidently classify your question."

In [64]:
import json
import pandas as pd
from langchain_core.documents import Document
from langchain_community.vectorstores import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA, LLMChain
from langchain_experimental.agents import create_pandas_dataframe_agent
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory


# -----------------------------
# Load & Prepare JSON Data
# -----------------------------
with open("result.json", "r") as f:
    data = json.load(f)

# Convert JSON to flat DataFrame
records = []
docs = []

for id_, entry in data.items():
    flat = {"ID": id_}
    flat.update(entry)
    records.append(flat)

    # For RAG docs
    text = f"ID: {id_}\n"
    for k, v in entry.items():
        if isinstance(v, list):
            text += f"{k}: {', '.join(map(str, v))}\n"
        else:
            text += f"{k}: {v}\n"
    docs.append(Document(page_content=text))

df = pd.DataFrame(records)

# -----------------------------
# LLM & Embeddings
# -----------------------------
llm = ChatGoogleGenerativeAI(model='gemini-1.5-flash')
embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# -----------------------------
# RAG Setup
# -----------------------------

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=50,  # Adjust based on model limits
    chunk_overlap=10  # Slight overlap for context continuity
)
split_docs = text_splitter.split_documents(docs)
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

vectorstore = Chroma.from_documents(split_docs, embedding, collection_name="ldp_docs")
retriever = vectorstore.as_retriever(search_kwargs={"k": 20})


from langchain.chains import StuffDocumentsChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain as CombineStuffDocumentsChain




# 🧠 Custom prompt
context_prompt = PromptTemplate.from_template("""
You are a data assistant helping analyze leadership assessment data.

Use the provided context to answer the question below. The context includes information on individuals, dashboards, and their skills.

Context:
{context}

Question:
{question}

Answer by combining insights across individuals. Focus on patterns, averages, and summaries. If no useful context is found, say "Not enough information."
""")

# 🔗 Chain
llm_chain = LLMChain(llm=llm, prompt=context_prompt)

stuff_chain = CombineStuffDocumentsChain(
    llm_chain=llm_chain,
    document_variable_name="context"
)

# 🧠 Updated RAG with context + memory
rag_chain = RetrievalQA(
    retriever=retriever,
    combine_documents_chain=stuff_chain,
    memory=memory
)

# -----------------------------
# Pandas Agent Setup
# -----------------------------
pandas_agent = create_pandas_dataframe_agent(
    llm=llm,
    df=df,
    verbose=True,
    allow_dangerous_code=True
)

# -----------------------------
# Classifier to Route Query
# -----------------------------
classifier_prompt = PromptTemplate.from_template("""
You are a smart classifier. Given a user question, decide if it should be handled using:

- "structured" → if it's about rankings, filters, math, comparisons, scores
- "semantic" → if it's about summaries, meaning, types of skills, descriptions

Return ONLY the word "structured" or "semantic".

Question: {query}
""")

classifier_chain = LLMChain(llm=llm, prompt=classifier_prompt)

# -----------------------------
# Unified Ask Function
# -----------------------------
def ask(query: str) -> str:
    route = classifier_chain.run({"query": query}).strip().lower()
    print(f"[Routing → {route}]")
    
    if route == "structured":
        return pandas_agent.run(query)
    elif route == "semantic":
        return rag_chain.run(query)
    else:
        return "Sorry, I couldn't confidently classify your question."



In [66]:
from IPython.display import Markdown

Markdown(ask("Which dashboard has the highest average LIS?"))


[Routing → structured]


> Entering new AgentExecutor chain...
Thought: I need to group the dataframe by 'Dashboard' and calculate the mean of the 'LIS' column for each group. Then, I need to find the dashboard with the maximum mean LIS.

Action: python_repl_ast
Action Input: 
```python
import pandas as pd
data = {'ID': ['LDP0394', 'LDP4315', 'LDP0127', 'LDP4802', 'LDP1313'], 
        'Dashboard': ['D12: Corporate Communications and Training', 'D4: Logistics and Site Management', 'D6: Procurement and Supply Chain Management', 'D5: Engineering and Technical Management', 'D13: Administrative and Support Services'], 
        'Position': ['CORPORATE CULTURE & TRAINING MANAGER', 'SITE MANAGER - CIVIL', 'PROCUREMENT MANAGER-OFFICE SUPPLY', 'COMMISSIONING MANAGER', 'SUPPORT SERVICES MANAGER'], 
        'Name': ['Donald  Finchum', 'Susan  Clark', 'John  Mendoza', 'Juan  Spegal', 'Ruth  Kingsley'], 
        'Email': ['d.finchum31@anondomain.com', 's.clark59@anondomain.com', 'j.mendoza41@anondom

D12: Corporate Communications and Training

In [67]:
mean_lis_by_dashboard = df.groupby('Dashboard')['LIS'].mean()
mean_lis_by_dashboard

Dashboard
D10: Financial and Risk Control                     80.650000
D11: Corporate Strategy and Business Development    86.844667
D12: Corporate Communications and Training          77.053800
D13: Administrative and Support Services            80.076000
D2: Executive Level Operations                      85.887500
D3: Senior Management - Operations and Logistics    75.324042
D4: Logistics and Site Management                   79.168694
D5: Engineering and Technical Management            76.401190
D6: Procurement and Supply Chain Management         77.378800
D7: Corporate Quality Assurance and Control         77.845500
D8: Health, Safety, and Security                    76.752000
D9: Corporate Governance and Compliance             66.360500
Name: LIS, dtype: float64

In [68]:
dashboard_with_highest_mean_lis = mean_lis_by_dashboard.idxmax()
dashboard_with_highest_mean_lis

'D11: Corporate Strategy and Business Development'

In [42]:
from IPython.display import Markdown

query = "What are the most common skills in D4?"
docs = retriever.get_relevant_documents(query)

# View retrieved context
for i, doc in enumerate(docs):
    print(f"\n--- Document {i+1} ---\n")
    print(doc.page_content)

# Then call the chain
Markdown(ask(query))


/var/folders/6v/qvykl7vd2yb8gqx3qb10rc5m0000gn/T/ipykernel_51521/2070393591.py:4: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(query)



--- Document 1 ---

ID: LDP4242
Dashboard: D4: Logistics and Site Management
Position: SITE MANAGER - CIVIL
Name: Katherine  Duggan
Email: k.duggan30@anondomain.com
LIS: 76.657
EQ: 108
Critical Skills: Problem-Solving, Project Management, Organization Skills, Accountability, Communication Skills
Critical Skills Scores: 90, 84, 79, 87, 92
Necessary Skills: Monitoring Employee Performance, Measuring & Evaluating Results, Conflict Management, Building Effective Teams
Necessary Skills Scores: 97, 59, 66, 76
Beneficial Skills: Negotiating Ability, Resilience/Hardiness, Innovative Mindset, Recruitment & Hiring, Supportiveness
Beneficial Skills Scores: 83, 72, 73, 73, 93
Key skills mean: 86.4
Necessary skills mean: 74.5
beneficial skills mean: 78.8


--- Document 2 ---

ID: LDP4242
Dashboard: D4: Logistics and Site Management
Position: SITE MANAGER - CIVIL
Name: Katherine  Duggan
Email: k.duggan30@anondomain.com
LIS: 76.657
EQ: 108
Critical Skills: Problem-Solving, Project Management, Organi

In Dashboard D4 (Logistics and Site Management), the most common skills across the two Site Managers (Civil and Mechanical) are:

* **Critical Skills:** Problem-Solving, Project Management, Organization Skills, Accountability, Communication Skills
* **Necessary Skills:** Monitoring Employee Performance, Measuring & Evaluating Results, Conflict Management, Building Effective Teams
* **Beneficial Skills:** Negotiating Ability, Resilience/Hardiness, Innovative Mindset, Recruitment & Hiring, Supportiveness

In [19]:
def get_insights_chart(lis_data, source_code, llm):
    insights_template = PromptTemplate(
        input_variables=["Dataframe", "PlotCode"],
        template="""You are a data analyst helping decision-makers understand data trends.

    Step 1: Analyze the PlotCode and DataFrame to understand what the chart is visualizing. Do NOT include or describe the code itself.

    Step 2: Return your output in **Markdown** using the following structure:

    **Chart Description**
    \n
    A brief paragraph that explains what the chart is visualizing and why it's relevant.

    **Chart Insights**
    - Use concise bullet points to highlight key patterns, trends, or outliers in the data.
    - Make the language natural and insightful, not robotic.

    **Chart Actionables**
    - Suggest meaningful next steps or considerations based on the data.
    - Actionables should feel strategic and helpful to someone using the chart to make decisions.

    Important Guidelines:
    - If a KPI, variable, or concept is not clearly defined in the DataFrame or PlotCode, do NOT guess its meaning. It may represent proprietary or internal data.
    - Never fabricate interpretations — only base your insights on what is explicitly observable from the data and plot logic.
    - Be clear, thoughtful, and avoid assumptions.

    Only return the final Markdown-formatted insight (no explanations, no code).

    DataFrame:
    {Dataframe}

    PlotCode:
    {PlotCode}
    """
    )

    #llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
    chain = insights_template | llm | StrOutputParser()
    response = chain.invoke({"Dataframe": lis_data, "PlotCode": source_code})
    return response



def build_donut_chart(lis_data):
    ''' 
    LIS Stands for Leadership Index Score which is a weighted score of critical skills necessary skills and beneficial skills to have a standardised metric to compare all individuals
    EQ assesses Emotional Intelligence which is one of the most important skills that all leaders are assessed in.
    '''
    mean_lis = np.mean(lis_data)
    std_lis = np.std(lis_data)
    std_low = mean_lis - 1.5 * std_lis
    std_high = mean_lis + 1.5 * std_lis

    leaders_meeting = np.sum((lis_data >= std_low) & (lis_data <= std_high))
    leaders_exceeding = np.sum(lis_data > std_high)
    leaders_requiring_training = np.sum(lis_data < std_low)

    labels = ['Meeting Minimum Competency', 'Exceeding Expectations', 'Requiring Training']
    values = [leaders_meeting, leaders_exceeding, leaders_requiring_training]
    colors = ['#5c9acc', '#f4a300', '#e63946']

    fig = go.Figure(data=[go.Pie(
        labels=labels,
        values=values,
        hole=0.4,
        marker=dict(colors=colors),
        hovertemplate="%{label}: %{value} leaders (%{percent})",
        textinfo='percent+label'
    )])
    fig.update_layout(
        title="Overall Leadership Competency Levels",
        margin=dict(t=50, b=50, l=50, r=50),
        width=700,
        height=500
    )
    return fig